In [ ]:
#Memsurfer analysis script for ENTH systems, computing curvature and other fun things

In [1]:
import os, sys
import numpy as np
import argparse
import logging
LOGGER = logging.getLogger(__name__)

import memsurfer
from memsurfer import utils

import MDAnalysis as mda
from MDAnalysis.analysis.distances import distance_array
from MDAnalysis.analysis.leaflet import LeafletFinder, optimize_cutoff

import mdreader
from lipidType import *


In [6]:
#example from paper, chart 1

#loading data
syst = mda.Universe('/u1/jessie.gan/5_ENTH/production0.gro','/u1/jessie.gan/5_ENTH/production0.xtc')

selFrame = [len(syst.trajectory)-1] # In this example, we only look at a single frame (the last one)
periodic = True # Our domain is periodic (in xy)
bbox = np.zeros((2,3)) 
bbox[1,:] = syst.dimensions[:3] # And this is the bounding box

print(syst)
print('dimensions = '+str(syst.dimensions[0:3]))


<Universe with 94331 atoms>
dimensions = [279.7987 279.7987 137.1732]


In [7]:
#defining lipids

lipidTypeList = lipid_masterlist() # Use a lipid master list to identify headgroups
lipidTypeNames = np.array([l[0] for l in lipidTypeList]) # Get list of all supported lipid types
resnames = np.unique(syst.atoms.resnames) # Get all resnames in this simulation
print('Resnames: '+resnames)

lipidTypeDic = {} # Make a lipid type dictionary
for i in np.where(np.in1d(lipidTypeNames, resnames))[0]:
    lipidTypeDic[lipidTypeNames[i]] = LipidType(lipidTypeList[i], syst)

# Get beads that define the heads of all flip-flopping non-leaflet-defining lipids
defFlipFlopHeadgroups = MDAnalysis.core.groups.AtomGroup([], syst)
for i in list(lipidTypeDic.values()):
    defFlipFlopHeadgroups += i.getNoneLeafletSelection(syst)

# Define top/bottom leaflets, Get beads that define the bilayer leaflets
# (not in bilayer middle, but closely packed, use linker + first tail bead of lipids that do not flip-flop)
defHeadgroups = MDAnalysis.core.groups.AtomGroup([], syst)
for i in list(lipidTypeDic.values()):
    defHeadgroups += i.getLeafletSelection(syst)
print(defHeadgroups)

# get leaflets
rcutoff, n = optimize_cutoff(syst, defHeadgroups)
lfls = LeafletFinder(syst, defHeadgroups, cutoff=rcutoff, pbc=True)
grps = lfls.groups()
top_head = grps[0] #top headgroups
bot_head = grps[1] #bottom headgroups
print('leaflet sizes: ' + str(len(top_head)) + ', ' + str(len(bot_head)))
rt = float(len(top_head))/len(bot_head)
print('Top-Bottom ratio: '+str(rt))


['Resnames: ALA' 'Resnames: ARG' 'Resnames: ASN' 'Resnames: ASP'
 'Resnames: CYS' 'Resnames: GLN' 'Resnames: GLU' 'Resnames: GLY'
 'Resnames: HIS' 'Resnames: ILE' 'Resnames: LEU' 'Resnames: LYS'
 'Resnames: MET' 'Resnames: PHE' 'Resnames: POPC' 'Resnames: POPS'
 'Resnames: PRO' 'Resnames: SER' 'Resnames: THR' 'Resnames: TRP'
 'Resnames: TYR' 'Resnames: VAL' 'Resnames: W']
<AtomGroup [<Atom 1791: NC3 of type N of resname POPC, resid 159 and segid SYSTEM>, <Atom 1803: NC3 of type N of resname POPC, resid 160 and segid SYSTEM>, <Atom 1815: NC3 of type N of resname POPC, resid 161 and segid SYSTEM>, ..., <Atom 31155: CNO of type C of resname POPS, resid 2606 and segid SYSTEM>, <Atom 31167: CNO of type C of resname POPS, resid 2607 and segid SYSTEM>, <Atom 31179: CNO of type C of resname POPS, resid 2608 and segid SYSTEM>]>
leaflet sizes: 1225, 1225
Top-Bottom ratio: 1.0


In [ ]:
for i in selFrame:
    # Get all lipids in top/bot leaflets (including flip-flop lipids - therefore has to be done for each frame)
    tp = top_head + defFlipFlopHeadgroups.select_atoms('around 12 fullgroup topsel', topsel=top_head)
    bt = bot_head + defFlipFlopHeadgroups.select_atoms('around 12 fullgroup botsel', botsel=bot_head)
    
    #computing membranes
    mt = memsurfer.Membrane.compute(tp.positions, tp.resnames, periodic = periodic, bbox = bbox)
    mb = memsurfer.Membrane.compute(bt.positions, bt.resnames, periodic = periodic, bbox = bbox)
    print('read!')

In [2]:
#example from github


if __name__ == '__main__':

    print ('using memsurfer from ({})'.format(memsurfer.__file__))

    # --------------------------------------------------------------------------
    # Path to input data.
    # Currently, we're using hardcoded data, but it can be passed as argument
    '''
    parser = argparse.ArgumentParser(description='Example script for MemSurfer')
    parser.add_argument('--traj', required=True, nargs=1, help='Trajectory file')
    parser.add_argument('--topo', required=True, nargs=1, help='Topology file')
    args = vars(parser.parse_args())
    '''
    args = dict()
    ddir = './data'
    args['traj'] = [os.path.join(ddir, '10us.35fs-DPPC.40-DIPC.30-CHOL.30.gro')]
    args['topo'] = [os.path.join(ddir, '10us.35fs-DPPC.40-DIPC.30-CHOL.30.tpr')]

    # this data has three lipids
    lipids = ['DIPC', 'DPPC', 'CHOL']

    # --------------------------------------------------------------------------
    # Create a logger
    memsurfer.utils.create_logger(2,1,0,'','')

    # The prefix we will use to name the output files
    outprefix = args['traj'][0]
    outprefix = outprefix[:outprefix.rfind('.')]

    # --------------------------------------------------------------------------
    # Use MDAnalysis to read the data
    fargs = ['-f', args['traj'][0], '-s', args['topo'][0]]
    LOGGER.info('arguments = {}'.format(fargs))

    syst = mdreader.MDreader(fargs)
    syst.add_argument('-sframe', metavar='SELFRAME', type=int, dest='selframe', default=-1, help='int \tframe to save')
    syst.do_parse()

    LOGGER.info('Number of frames in sim {}'.format(len(syst)))
    LOGGER.info('System dimensions: {}'.format(syst.dimensions[0:3]))

    # In this example, we only look at a single frame (the last one)
    selFrame = [len(syst)-1]

    # Our domain is periodic (in xy)
    periodic = True

    # And this is the bounding box
    bbox=np.zeros((2,3))
    bbox[1,:] = syst.dimensions[:3]
    
    # --------------------------------------------------------------------------
    # Use a lipid master list to identify headgroups
    lipidTypeList = lipid_masterlist()

    # Get list of all suported lipid types
    lipidTypeNames = np.array([l[0] for l in lipidTypeList])

    # Get all resnames in this simulation
    resnames = np.unique(syst.atoms.resnames)

    # Make a lipid type dictionary
    LOGGER.warning('Following resnames either not lipids or not supported {}'
                    .format(resnames[np.logical_not(np.in1d(resnames, lipidTypeNames))]))

    lipidTypeDic = {}
    for i in np.where(np.in1d(lipidTypeNames, resnames))[0]:
        lipidTypeDic[lipidTypeNames[i]] = LipidType(lipidTypeList[i], syst)

    # Get beads that define the heads of all flip-flopping none leaflet defining lipids
    defFlipFlopHeadgroups = MDAnalysis.core.groups.AtomGroup([], syst)

    for i in list(lipidTypeDic.values()):
        defFlipFlopHeadgroups += i.getNoneLeafletSelection(syst)

    #LOGGER.info('FlipFlopHeadgroups = {}'.format(defFlipFlopHeadgroups))

    # --------------------------------------------------------------------------
    # Define top/bottom leaflets

    # Get beads that define the bilayer leaflets
    # (not in bilayer middle, but closely packed, use linker + first tail bead of lipids that do not flip-flop)
    defHeadgroups = MDAnalysis.core.groups.AtomGroup([], syst)
    for i in list(lipidTypeDic.values()):
        defHeadgroups += i.getLeafletSelection(syst)
    
    # get leaflets
    rcutoff, n = optimize_cutoff(syst, defHeadgroups)
    lfls = LeafletFinder(syst, defHeadgroups, cutoff=rcutoff, pbc=True)

    grps = lfls.groups()

    if len(grps) == 2:
        LOGGER.info('Found {} leaflet groups'.format(len(grps)))

    # check if they're even
    top_head = grps[0]
    bot_head = grps[1]

    rt = float(len(top_head))/len(bot_head)
    if rt > 1.3 or rt < 0.77:
        raise ValueError('Found uneven leaflets. top = {}, bot = {}'.format(len(top_head), len(bot_head)))

    LOGGER.info('Found leaflets of size {}, {}'.format(len(top_head), len(bot_head)))
    

2020-08-29 17:07:16,350 [INFO] __main__:<module> - arguments = ['-f', './data/10us.35fs-DPPC.40-DIPC.30-CHOL.30.gro', '-s', './data/10us.35fs-DPPC.40-DIPC.30-CHOL.30.tpr']
Loading...
2020-08-29 17:07:16,364 [INFO] MDAnalysis.topology.TPRparser:_log_header - Gromacs version   : b'VERSION 5.1.4'
2020-08-29 17:07:16,365 [INFO] MDAnalysis.topology.TPRparser:_log_header - tpx version       : 103
2020-08-29 17:07:16,367 [INFO] MDAnalysis.topology.TPRparser:_log_header - tpx generation    : 26
2020-08-29 17:07:16,368 [INFO] MDAnalysis.topology.TPRparser:_log_header - tpx precision     : 4
2020-08-29 17:07:16,369 [INFO] MDAnalysis.topology.TPRparser:_log_header - tpx file_tag      : b'release'
2020-08-29 17:07:16,370 [INFO] MDAnalysis.topology.TPRparser:_log_header - tpx natoms        : 110376
2020-08-29 17:07:16,370 [INFO] MDAnalysis.topology.TPRparser:_log_header - tpx ngtc          : 2
2020-08-29 17:07:16,371 [INFO] MDAnalysis.topology.TPRparser:_log_header - tpx fep_state     : 0
2020-08-2

using memsurfer from (/opt/miniconda3/lib/python3.7/site-packages/memsurfer-1.0.0-py3.7-linux-x86_64.egg/memsurfer/__init__.py)


/opt/miniconda3/lib/python3.7/site-packages/MDAnalysis/coordinates/base.py:837: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")
2020-08-29 17:07:17,523 [INFO] __main__:<module> - Number of frames in sim 1
2020-08-29 17:07:17,525 [INFO] __main__:<module> - System dimensions: [273.4512  273.4512  153.51651]
2020-08-29 17:07:17,573 [WARNING] __main__:<module> - Following resnames either not lipids or not supported ['ION' 'W']
2020-08-29 17:07:18,938 [INFO] __main__:<module> - Found 2 leaflet groups
2020-08-29 17:07:18,939 [INFO] __main__:<module> - Found leaflets of size 1064, 1064


In [ ]:
    # --------------------------------------------------------------------------
    # Compute membranes for all frames
    for i in selFrame:

        print('\n')
        # Select frame 0 to len(syst)
        ts = syst.trajectory[i]
        LOGGER.info('Frame: %5d, Time: %8.3f ps' % (ts.frame, syst.trajectory.time))

        # Get all lipids in top/bot leaflets (including flip-flop lipids - therefore has to be done for each frame)
        tp = top_head + defFlipFlopHeadgroups.select_atoms('around 12 fullgroup topsel', topsel=top_head)
        bt = bot_head + defFlipFlopHeadgroups.select_atoms('around 12 fullgroup botsel', botsel=bot_head)

        if len(tp.select_atoms('group bt', bt=bt)):
            errmsg = 'Frame {}: {} common atoms between leaflets.'.format(syst.trajectory.frame, len(tp.select_atoms('group bt', bt=bt)))
            LOGGER.warning(errmsg)
            raise ValueError(errmsg)

        ## Select one bead per lipid (could do this better by getting the residues - and selecting e.g. first of each
        LOGGER.info('We have {} lipids in upper and {} in lower leaflets'.format(len(tp), len(bt)))

        # ----------------------------------------------------------------------
        # This is where we use MemSurfer
        # ----------------------------------------------------------------------
        mt = memsurfer.Membrane.compute(tp.positions, tp.resnames, bbox, periodic)
        mb = memsurfer.Membrane.compute(bt.positions, bt.resnames, bbox, periodic)

        # compute total density
        sigmas = [10,20,30,40]
        get_nlipdis = True
        memsurfer.Membrane.compute_densities([mt, mb], [2], sigmas, get_nlipdis, 'all')

        # compute density of each type of lipid
        for l in lipids:
            memsurfer.Membrane.compute_densities([mt, mb], [2], sigmas, get_nlipdis, 'all')

        if True:
            # compute density of each type of lipid
            mt.write_all(outprefix+'_f{}-top'.format(ts.frame),
                    {'frame': ts.frame, 'time': syst.trajectory.time})

            mb.write_all(outprefix+'_f{}-bot'.format(ts.frame),
                    {'frame': ts.frame, 'time': syst.trajectory.time})

        # --------------------------------------------------------------------------
        # --------------------------------------------------------------------------

/opt/miniconda3/lib/python3.7/site-packages/MDAnalysis/coordinates/base.py:837: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")
2020-08-29 17:07:27,825 [INFO] __main__:<module> - Frame:     0, Time:    0.000 ps
/opt/miniconda3/lib/python3.7/site-packages/MDAnalysis/core/selection.py:261: DeprecationWarning: `fullgroup` is deprecated, use `global group` instead!
 This will be removed in v0.15.0
  sel = self.sel.apply(group)
2020-08-29 17:07:27,835 [INFO] __main__:<module> - We have 1515 lipids in upper and 1502 in lower leaflets
2020-08-29 17:07:27,836 [INFO] memsurfer.membrane:__init__ - Initializing Membrane with (1515, 3) points
2020-08-29 17:07:27,837 [INFO] memsurfer.membrane:__init__ - 	 actual bbox   = [6.0000002e-02 1.0000001e-02 3.8720001e+01] [273.29 273.12  88.14]
2020-08-29 17:07:27,838 [INFO] memsurfer.membrane:__init__ - 	 given periodic bbox = [0. 0. 0.] [273.45120239 273.45120239 153.51651001]
2020-0